# Music Generation With Machine Learning

## All nescessary imports for the project

In [397]:
import music21 as m21
import os
import tensorflow.keras as keras
import numpy as np
import json
m21.environment.set('musescoreDirectPNGPath', '/Applications/MuseScore 4.app/Contents/MacOS/mscore')

## Global parameters

In [391]:
SONG_DIR = 'erk'
FILE_EXSTENSION = 'krn'
NOTE_LENGTHS = [i*0.25 for i in range(1, 17)]
SEQUENCE_SIZE = 100

## Functions

### Read songs from directory

In [372]:
def read_songs_from_dir(song_dir, file_exstension):
    """
    Returns a list of all songs with correct file exstension from a directory
    
    Input params
    song_dir: Directory for the songs
    file_exstension: The fileformat for the songs (in case of more files in directory)
    
    Returns
    songs: List of all songs as music21 score objects
    """
    songs = []
    for path, subdirs, files in os.walk(song_dir):
        for file in [i for i in files if i[-3:] == file_exstension]:
            song = m21.converter.parse(path+'/'+ file)
            songs.append(song)
    return songs

### Check if all notes in a song are of the acceptable length

In [373]:
def valid_length(song, note_length):
    '''
    Check if all notes in a song are of the acceptable length.
    
    Input params
    song: Song as music 21 object
    note_length: List of acceptable note length in quarternotes
    
    Returns:
    True if all notes are correct length
    False if any note is not correct length
    '''
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in (note_length):
            return False
    return True

### Get key of song

In [374]:
def get_key(song):
    '''
    Function that returns the key of a song, if not found, then calculated instead.
    
    Input params
    song: Song as music 21object
    
    Returns
    key: Key as music21 object
    '''
    parts = song.getElementsByClass(m21.stream.Part)
    measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measures_part0[0][4]
    if not isinstance(key, m21.key.Key):
        key = song.analyze("key")
    return key

### Transpose song to C/Am

In [375]:
def transpose_song(song, key):
    '''
    Transposes song to C if major, Am if minor
    
    Input params:
    song: Song as music21 object
    key: Key as music21 object
    
    Returns:
    song_transposed: Song in C major or A minor as music21 object
    '''
    if key.mode == 'minor':
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch('A'))
    else:
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch('C'))
    song_transposed = song.transpose(interval)
    return song_transposed

### Converts song into a time series

In [376]:
def song_to_time_series(song):
    '''
    Adds a symbol for every pitch or rest. Depending of the duration of the note/rest, '-' are added 
    for every 16th note it's ringing.
    
    Input params
    song: Song as music21 object
    
    Returns
    song_time_series: List with all songs as time series
    '''
    song_time_series = []
    for note in song.flat.notesAndRests:
        if isinstance(note, m21.note.Note):
            symbol = note.pitch.midi
        elif isinstance(note, m21.note.Rest):
            symbol = 'R'
        length = int(note.duration.quarterLength/0.25)
        song_time_series.append(symbol)
        if length > 1:
            for i in range(length-1):
                song_time_series.append('-')
    return song_time_series

### Map the notes to integers and save that dict

In [377]:
def mapping_notes(songs_time_series):
    '''
    Maps all the elements in the time series list to integers, and saves that dict
    
    Input params
    songs_time_series: List of songs in time series format
    
    Returns
    mapping: Mapping as dictionary
    '''
    songs = [note for song in songs_time_series for note in song]
    notes = list(set(songs))
    mapping = {}
    for i, note in enumerate(notes):
        mapping[note] = i
    with open('mapping_file', "w") as fp:
        json.dump(mapping, fp, indent=4)
    return mapping

### Creates a new list of mapped songs

In [378]:
def map_songs(songs_time_series, mapping):
    '''
    Converts the time series list of songs into a list of mapped songs
    
    Input params
    songs_time_series: list of songs in time series format
    mapping: dictionary with every note in songs_time_series mapped to an int
    
    Returns
    mapped_songs: list of songs mapped
    '''
    mapped_songs = []
    for song in songs_time_series:
        mapped_song = []
        for note in song:
            mapped_note = mapping[note]
            mapped_song.append(mapped_note)
        mapped_songs.append(mapped_song)
    return mapped_songs

### Creates training data as inputs and target

In [379]:
def create_input_target_data(mapped_songs, sequence_size):
    '''
    Takes the mapped songs and a sequence size to create inputs sequences and target values. Target value would
    be the note after the sequence.
    
    Input params
    mapped_songs: list of mapped songs
    sequence_size: int with length of training sequences
    
    Returns
    X: Sequence of notes with sequence_size length
    y: Target for sequence
    '''
    inputs = []
    target = []
    for song in mapped_songs:
        for i in range(len(song) - sequence_size):
            inputs.append(song[i:i + sequence_size])
            target.append(song[i + sequence_size])
    X = keras.utils.to_categorical(inputs, num_classes=len(mapping))
    y = np.array(target)
    return X, y

### Creates lstm model

In [398]:
def create_model(nbr_outputs,nbr_units):
    '''
    Creates a three hidden layer model with 2 lstm and one dropout layer. Uses accuracy for metrics, 
    sparse_categorical_crossentropy for loss, Adam optimizer, and activation function softmax.
    
    Input params
    nbr_outputs: The number of outputs the model should have
    nbr_units: The number of units for the lstm layers
    
    Returns
    model: Keras Tensorflow neural network model
    '''
    inputs = keras.layers.Input(shape=(None, nbr_outputs))
    lstm1 = keras.layers.LSTM(nbr_units, return_sequences=True)(inputs)
    lstm2 = keras.layers.LSTM(nbr_units)(lstm1)
    dropout = keras.layers.Dropout(0.2)(lstm2) 
    output = keras.layers.Dense(nbr_outputs, activation="softmax")(dropout)

    model = keras.Model(inputs, output)

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001),
                  metrics=["accuracy"])

    model.summary()
    return model

## Main

In [392]:
songs_time_series = []
for song in read_songs_from_dir(SONG_DIR, FILE_EXSTENSION):
    if valid_length(song, NOTE_LENGTHS):
        key = get_key(song)
        transposed_song = transpose_song(song, key)
        song_time_series = song_to_time_series(transposed_song)
        songs_time_series.append(song_time_series)
mapping = mapping_notes(songs_time_series)
mapped_songs = map_songs(songs_time_series, mapping)
X_train, y_train = create_input_target_data(mapped_songs, SEQUENCE_SIZE)

## Creating the model

In [393]:
model = create_model(len(mapping), 512)

Model: "model_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_60 (InputLayer)       [(None, None, 21)]        0         
                                                                 
 lstm_79 (LSTM)              (None, None, 512)         1093632   
                                                                 
 lstm_80 (LSTM)              (None, 512)               2099200   
                                                                 
 dropout_54 (Dropout)        (None, 512)               0         
                                                                 
 dense_84 (Dense)            (None, 21)                10773     
                                                                 
Total params: 3203605 (12.22 MB)
Trainable params: 3203605 (12.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Training the model and saving it

In [395]:
model.fit(X_train, y_train, epochs=50, batch_size = 64)
model.save('model_lstm')

Epoch 1/50
72/72 [==============================] - 49s 665ms/step - loss: 1.4592 - accuracy: 0.6651
Epoch 2/50
72/72 [==============================] - 47s 646ms/step - loss: 1.3139 - accuracy: 0.6773
Epoch 3/50
72/72 [==============================] - 47s 654ms/step - loss: 1.2247 - accuracy: 0.6747
Epoch 4/50
72/72 [==============================] - 47s 651ms/step - loss: 1.0217 - accuracy: 0.6729
Epoch 5/50
72/72 [==============================] - 47s 657ms/step - loss: 0.9823 - accuracy: 0.6812
Epoch 6/50
72/72 [==============================] - 46s 644ms/step - loss: 0.9572 - accuracy: 0.6840
Epoch 7/50
72/72 [==============================] - 46s 640ms/step - loss: 0.9313 - accuracy: 0.6941
Epoch 8/50
72/72 [==============================] - 46s 641ms/step - loss: 0.9046 - accuracy: 0.6989
Epoch 9/50
72/72 [==============================] - 47s 652ms/step - loss: 0.8770 - accuracy: 0.7045
Epoch 10/50
72/72 [==============================] - 47s 651ms/step - loss: 0.8309 - accura

INFO:tensorflow:Assets written to: model_4/assets


## Functions for complex music

### Read songs from dir where song has many instruments

In [405]:
def read_songs_from_dir_parts(song_dir, file_exstension):
    """
    Returns a list of all songs as parts with correct file exstension from a directory
    
    Input params
    song_dir: Directory for the songs
    file_exstension: The fileformat for the songs (in case of more files in directory)
    
    Returns
    songs: List of all songs as music21 parts objects
    """
    songs = []
    for path, subdirs, files in os.walk(song_dir):
        for file in [i for i in files if i[-3:] == file_exstension]:
            song = m21.converter.parse(path+'/'+ file)
            parts = m21.instrument.partitionByInstrument(song)
            songs.append(parts[0].recurse())         
    return songs

### Get key for complex music

In [400]:
def get_key_complex(song):
    '''
    Function that returns the key of a song, if not found, then calculated instead.
    
    Input params
    song: Song as music21 part object
    
    Returns
    key: Key as music21 object
    '''
    for part in song:
        if isinstance(part, m21.key.Key):
            return part
        elif isinstance(part, m21.note.Note):
            return song.analyze("key")

### Two functions that converts the complex durations to 16th note dividable notes

In [402]:
def find_closest_value(target, values):
    '''
    Finds the closest value in a list to the target
    
    Input params
    target: number to find the closest to in list
    values: list of numbers
    
    Returns
    closest_value: Closest value from the target in the list values
    '''
    closest_value = min(values, key=lambda x: abs(x - target))
    return closest_value

def convert_duration(song, note_length):
    '''
    Converts the duration of a note in a song, to the closest one in note_length list.
    
    Input params
    song: song as music21 object
    note_length: list of acceptable note lenghts
    
    Returns
    song: song as music21 object with updated durations
    '''
    for note in song.flatten().notesAndRests:
        if note.duration.quarterLength not in (note_length):
            note.duration.quarterLength = find_closest_value(note.duration.quarterLength, note_length)
    return song

### Two functions to convert song to time series data, can handle chords

In [410]:
def simplify_chord(chord):
    '''
    Returns the highest note in the chord
    
    Input params
    chord: chord as music21 object Chord
    
    Returns: highest pitch in chord in midi format
    '''
    return [note.pitch.midi for note in chord.notes][-1]

def song_to_time_series_complex(song):
    '''
    Adds a symbol for every pitch or rest. Depending of the duration of the note/rest, '-' are added 
    for every 16th note it's ringing. Also handles chords
    
    Input params
    song: Song as music21 object
    
    Returns
    song_time_series: List with all songs as time series
    '''
    song_time_series = []
    for note in song.flatten().notesAndRests:
        if isinstance(note, m21.note.Note):
            symbol = note.pitch.midi
        elif isinstance(note, m21.note.Rest):
            symbol = 'R'
        elif note.isChord:
            symbol = simplify_chord(note)
        length = int(note.duration.quarterLength/0.25)
        song_time_series.append(symbol)
        if length > 1:
            for i in range(length-1):
                song_time_series.append('-')
    return song_time_series

## Global parameters for complex songs

In [413]:
SONG_DIR = 'midi_songs'
FILE_EXSTENSION = 'mid'
NOTE_LENGTHS = [i*0.25 for i in range(1, 17)]#['1/3','2/3','1/12','1/6','5/12' ,'4/3'
                #, '0.25', '0.5', '0.75', '1.0', '1.25', '1.5', '2.0', '3.0', '4.0']
SEQUENCE_SIZE = 64

## Main

In [416]:
songs_time_series = []
for song in read_songs_from_dir_parts(SONG_DIR, FILE_EXSTENSION):
    key = get_key_complex(song)
    if key == None:
        continue
    transposed_song = transpose_song(convert_duration(song,NOTE_LENGTHS), key)
    song_time_series = song_to_time_series_complex(transposed_song)
    songs_time_series.append(song_time_series)
mapping = mapping_notes(songs_time_series)
mapped_songs = map_songs(songs_time_series, mapping)
X_train, y_train = create_input_target_data(mapped_songs, SEQUENCE_SIZE)

/var/folders/g8/7_44w75n3s312twwsd2rdyyh0000gn/T/ipykernel_85199/3702378589.py:26: StreamIteratorInefficientWarning: flatten is not defined on StreamIterators. Call .stream() first for efficiency
  for note in song.flatten().notesAndRests:
/var/folders/g8/7_44w75n3s312twwsd2rdyyh0000gn/T/ipykernel_85199/219652541.py:6: StreamIteratorInefficientWarning: transpose is not defined on StreamIterators. Call .stream() first for efficiency
  song_transposed = song.transpose(interval)
/var/folders/g8/7_44w75n3s312twwsd2rdyyh0000gn/T/ipykernel_85199/288021183.py:6: StreamIteratorInefficientWarning: analyze is not defined on StreamIterators. Call .stream() first for efficiency
  return song.analyze("key")


## Creating the model

In [417]:
model = create_model(len(mapping), 400)

Model: "model_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_61 (InputLayer)       [(None, None, 86)]        0         
                                                                 
 lstm_81 (LSTM)              (None, None, 400)         779200    
                                                                 
 lstm_82 (LSTM)              (None, 400)               1281600   
                                                                 
 dropout_55 (Dropout)        (None, 400)               0         
                                                                 
 dense_85 (Dense)            (None, 86)                34486     
                                                                 
Total params: 2095286 (7.99 MB)
Trainable params: 2095286 (7.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train and save the model

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size = 64)
model.save('model_complex')